- This notebook
- Identifies all the document ids to obtain from the EDINET API
- Leaves only xbrl and xsd files

In [1]:
import pandas as pd
from sqlalchemy import create_engine, Column, String, Integer, MetaData, Table
from sqlalchemy.dialects.postgresql import NUMERIC, BIGINT
import psycopg2
from bs4 import BeautifulSoup
import zipfile
import io
import requests
import pandas as pd
from tqdm.notebook import tqdm
import time
import glob
import re
import json

# load const
with open('parameters.json', 'r') as file:
    data = json.load(file)

query = """
select x."Code_x", y."ＥＤＩＮＥＴコード", z.* 
from t_short_list x 
inner join t_edinet_code_mappings y on x."Code_x" = y."証券コード" 
inner join t_doc_list z on y."ＥＤＩＮＥＴコード" = z."edinetCode"
"""

query2 = """
SELECT 
    * 
FROM (
    SELECT
        Y2.*,
        Y3."ingestedDocID"
    FROM (
        SELECT 
            X."Code_x",
            X."ＥＤＩＮＥＴコード",
            MAX("periodEnd") AS latest_period_end
        FROM (
            SELECT 
                x."Code_x", 
                y."ＥＤＩＮＥＴコード", 
                z.* 
            FROM 
                t_short_list x 
            INNER JOIN 
                t_edinet_code_mappings y 
                ON x."Code_x" = y."証券コード" 
            INNER JOIN 
                t_doc_list z 
                ON y."ＥＤＩＮＥＴコード" = z."edinetCode"
        ) X
        GROUP BY 
            X."Code_x", 
            X."ＥＤＩＮＥＴコード"
    ) Y1 
    INNER JOIN
        t_doc_list Y2 
        ON Y1."ＥＤＩＮＥＴコード" = Y2."edinetCode" 
        AND Y1.latest_period_end = Y2."periodEnd"
    LEFT OUTER JOIN (
        SELECT DISTINCT "docID" AS "ingestedDocID" 
        FROM t_financials
    ) Y3 
    ON Y2."docID" = Y3."ingestedDocID"
) XYZ
WHERE
    XYZ."ingestedDocID" IS NULL;

"""

# Define the database connection string (update with your PostgreSQL details)
DATABASE_URI = data["DATABASE_URI"]
EDINET_KEY = data["EDINET_KEY"]

# Create an engine and metadata object
engine = create_engine(DATABASE_URI)

In [5]:
class GetReportFromEdinet:

    def __init__(self,docid_list):
        self.docid_list = docid_list

    def get_xbrl_file(self):
        for docid in tqdm(self.docid_list):
            # 書類取得APIのエンドポイント
            url = "https://api.edinet-fsa.go.jp/api/v2/documents/" + docid
#            print(url)
            time.sleep(5)


            #書類取得APIのパラメータ
            params ={"type":1,"Subscription-Key":EDINET_KEY}
            res = requests.get(url,params=params, verify=False)

            #ファイルへの出力
#            print(res.status_code)
            if res.status_code == 200:
            # レスポンスからZIPファイルを読み込む
                try:
                    with zipfile.ZipFile(io.BytesIO(res.content)) as z:
                    # ZIPファイル内のすべてのファイルをループ処理
                        for file in z.namelist():
                            if file.startswith("XBRL/PublicDoc/") and (file.endswith(".xsd") or file.endswith(".xbrl")):
                            # .xbrlもしくは.xsdファイルを見つけたら、それをディスクに書き込む
                                z.extract(file, path=f'./content/{docid}/')
                except:
                    continue

In [7]:
df = pd.read_sql(query2, engine)
docid_list = df["docID"].values
grfe = GetReportFromEdinet(docid_list)
grfe.get_xbrl_file()

  0%|          | 0/1 [00:00<?, ?it/s]

/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.edinet-fsa.go.jp'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
